## Metrics Calculations Tutorial

This notebook provides examples on how to carry out data metrics calcuations and analysis using the post_processing python library. Be sure to go through the [Quick Start](https://nhs-postprocessing.readthedocs.io/en/stable/QuickStart.html) section of the [documentation](https://nhs-postprocessing.readthedocs.io/en/stable/index.html) for instructions on how to access and import the libary and its packages.

If you would like to open an editable runnable version of the tutorial click [here](https://mybinder.org/v2/gh/UchechukwuUdenze/NHS_PostProcessing/main?%2FHEAD=&urlpath=%2Fdoc%2Ftree%2Fdocs%2Fsource%2Fnotebooks%2Ftutorial-metrics.ipynb) to be directed to a binder platform

<mark>The Library is still under active development and empty sections will be completed in Due time</mark>

### Table of content
- [Available Metrics](#available-metrics)
- [Single Data Metrics](#single-data-metrics)
- [Comparison Metrics](#comparison-metrics)

 All files are available in the github repository [here](https://github.com/UchechukwuUdenze/NHS_PostProcessing/tree/main/docs/source/notebooks)

### Requirements

The conda environmnent contains all libraries associated the post processing library. After setting up the conda environment, you only have to import the metrics maniupulation module from postprocessinglib.evaluation.

In [1]:
### Remove and modify these later.
import sys
import pandas as pd
sys.path.append("../../../")

In [2]:
from postprocessinglib.evaluation import data, metrics

Lets use one of the data blocks from the data manipulation tutorial

In [3]:
# passing a controlled csv file for testing
path_output = "MESH_output_streamflow_3.csv"
path_input = "Station_data.xlsx"

DATAFRAMES = data.generate_dataframes(csv_fpaths=path_output, warm_up=91)
               
Stations = pd.read_excel(io=path_input)

ignore = []
for i in range(0, len(Stations)):
    if Stations['Properties'][i] == 'X':
        ignore.append(i)

Stations = Stations.drop(Stations[Stations['Properties'] == 'X'].index)
Stations = Stations.set_index('Station Number')

for i in reversed(ignore):
        DATAFRAMES["DF_OBSERVED"] = DATAFRAMES["DF_OBSERVED"].drop(columns = DATAFRAMES['DF_OBSERVED'].columns[i])
        DATAFRAMES['DF_SIMULATED']  = DATAFRAMES["DF_SIMULATED"].drop(columns = DATAFRAMES['DF_SIMULATED'].columns[i])
        for key, dataframe in DATAFRAMES.items():
            if key != "DF_SIMULATED" and key != "DF_OBSERVED" and key != "NUM_SIMS":
                DATAFRAMES[key] = dataframe.drop(columns = dataframe.columns[[2*i, 2*i+1]])
            

# for key, value in DATAFRAMES.items():
#     print(f"{key}:\n{value.head}")

The start date for the Data is 1982-01-01


Now that we have our data, let's jump right in!

### Available Metrics

Because the library is in active development, there will be regular removals and additions to its features. As a rule of thumb therefore it is always a good idea to check what it can do at the time of use. We can do this by going ->

In [4]:
metrics.available_metrics()

['MSE - Mean Square Error',
 'RMSE - Roor Mean Square Error',
 'MAE - Mean Average Error',
 'NSE - Nash-Sutcliffe Efficiency ',
 'NegNSE - Nash-Sutcliffe Efficiency * -1',
 'LogNSE - Log of Nash-Sutcliffe Efficiency',
 'NegLogNSE - Log of Nash-Sutcliffe Efficiency * -1',
 'KGE - Kling-Gupta Efficiency',
 'NegKGE - Kling-Gupta Efficiency * -1',
 'KGE 2012 - Kling-Gupta Efficiency modified as of 2012',
 'BIAS- Prcentage Bias',
 'AbsBIAS - Absolute Value of the Percentage Bias',
 'TTP - Time to Peak',
 'TTCoM - Time to Centre of Mass',
 'SPOD - Spring Pulse ONset Delay',
 'FDC Slope - Slope of the Flow Duration Curve']

### Single Data Metrics
These are the metrics that only apply to just one of either the simulated or observed data. They are less about analysis and more about obtaining information about the data. These aren't made to compare but rather to inform trends and behaviours at a particular station. The library has 4 of them :

- [Time to Peak](#time-to-peak)
- [Time to Centre of Mass](#time-to-centre-of-mass)
- [Spring Pulse Onset Delay](#spring-pulse-onset-delay)
- [Slope of the Flow Duration Curve](#flow-duration-curve-slope)

#### Time to Peak
This helps to show how long it takes on average to get to the highest streamflow each year. An example is shown below:

In [5]:
# The Time to Peak for the simulated data will look like 
print(metrics.time_to_peak(df=DATAFRAMES['DF_SIMULATED']))

# The time to peak for the observed data looks like:-
print(metrics.time_to_peak(df=DATAFRAMES['DF_OBSERVED']))

              ttp
Station 1   171.0
Station 2   177.0
Station 3   176.0
Station 4   169.0
Station 5   169.0
Station 6   174.0
Station 7   166.0
Station 8   157.0
Station 9   157.0
Station 10  168.0
Station 11  179.0
Station 12  162.0
Station 13  172.0
Station 14  171.0
Station 15  171.0
Station 16  175.0
Station 17  168.0
Station 18  188.0
Station 19  187.0
Station 20  184.0
Station 21  187.0
Station 22  174.0
Station 23  173.0
Station 24  207.0
Station 25  175.0
Station 26  184.0
Station 27  147.0
Station 28  148.0
Station 29  151.0
Station 30  184.0
Station 31  141.0
Station 32  145.0
Station 33  160.0
Station 34  176.0
Station 35  177.0
Station 36  165.0
Station 37  208.0
Station 38  177.0
Station 39  147.0
Station 40  158.0
              ttp
Station 1   157.0
Station 2   157.0
Station 3   158.0
Station 4   159.0
Station 5   160.0
Station 6   172.0
Station 7   175.0
Station 8   166.0
Station 9   165.0
Station 10  173.0
Station 11  189.0
Station 12  169.0
Station 13  164.0
Station 14

As you can see, at the first station, on average, over the years, the highest predicted streamflow value will usually occur after 170 days - somewhere in the third week of June. For the second station on average, over the years, the highest predicted streamflow value usually occur after 177 days - somewhere in the final week of June. 
As you can see, you are able to observe and notice trends with the data at specific stations.

#### Time to Centre of Mass
This helps to show how long it takes on average to obtain 50% of the streamflow each year. An example is shown below:

In [6]:
# The Time to Centre of Mass for the simulated data will look like 
print(metrics.time_to_centre_of_mass(df=DATAFRAMES['DF_SIMULATED']))

# The time to Centre of Mass for the observed data looks like:-
print(metrics.time_to_centre_of_mass(df=DATAFRAMES['DF_OBSERVED']))

            ttcom
Station 1   185.0
Station 2   166.0
Station 3   188.0
Station 4   183.0
Station 5   186.0
Station 6   186.0
Station 7   190.0
Station 8   183.0
Station 9   181.0
Station 10  158.0
Station 11  175.0
Station 12  185.0
Station 13  170.0
Station 14  182.0
Station 15  182.0
Station 16  184.0
Station 17  187.0
Station 18  176.0
Station 19  190.0
Station 20  190.0
Station 21  193.0
Station 22  187.0
Station 23  193.0
Station 24  205.0
Station 25  190.0
Station 26  190.0
Station 27  155.0
Station 28  150.0
Station 29  163.0
Station 30  189.0
Station 31  163.0
Station 32  167.0
Station 33  172.0
Station 34  173.0
Station 35  188.0
Station 36  170.0
Station 37  193.0
Station 38  189.0
Station 39  156.0
Station 40  185.0
            ttcom
Station 1     NaN
Station 2     NaN
Station 3     NaN
Station 4   178.0
Station 5     NaN
Station 6   178.0
Station 7     NaN
Station 8   194.0
Station 9   186.0
Station 10  172.0
Station 11    NaN
Station 12  186.0
Station 13  177.0
Station 14

As you can see, at the fourth station, on average, over the years, 50% of the total volume of streamflow each year will usually have occured by 178 days - somewhere in the final week of June and for the twentieth station, after 179 days - Right at the end of June. 

#### Spring Pulse Onset Delay
This is used to determine what day snowmelt starts. An example is shown below:

In [7]:
# The Spring Pulse Onset for the simulated data will look like 
print(metrics.SpringPulseOnset(df=DATAFRAMES['DF_SIMULATED']))

# The Spring Pulse Onset for the observed data looks like:-
print(metrics.SpringPulseOnset(df=DATAFRAMES['DF_OBSERVED']))

             SPOD
Station          
Station 1   128.0
Station 2   115.0
Station 3   127.0
Station 4   119.0
Station 5   122.0
Station 6   124.0
Station 7   140.0
Station 8   128.0
Station 9   126.0
Station 10  296.0
Station 11  113.0
Station 12  136.0
Station 13  110.0
Station 14  124.0
Station 15  126.0
Station 16  129.0
Station 17  128.0
Station 18  123.0
Station 19  143.0
Station 20  145.0
Station 21  145.0
Station 22  123.0
Station 23  120.0
Station 24  198.0
Station 25  137.0
Station 26  121.0
Station 27  109.0
Station 28   98.9
Station 29  113.0
Station 30  121.0
Station 31  120.0
Station 32  116.0
Station 33  112.0
Station 34  114.0
Station 35  117.0
Station 36  134.0
Station 37  185.0
Station 38  146.0
Station 39  103.0
Station 40  111.0
             SPOD
Station          
Station 1   113.0
Station 2     NaN
Station 3   101.0
Station 4   114.0
Station 5   115.0
Station 6   124.0
Station 7   142.0
Station 8   136.0
Station 9   138.0
Station 10  296.0
Station 11  172.0
Station 12

This shows us that at the first station, on average, over the years, snowmelt is predicted to begin 127 days into the year - somewhere in the First week of May. For the third station on average, over the years, snowmelt is predicted to begin 126 days into the year - somewhere in the First week of May as well

#### Flow Duration Curve Slope
This is used to calculate the slope of the flow duration curve. An example is shown below:

In [8]:
# The Fliw Duration Curve for the Simulated Data will look like 
print(metrics.slope_fdc(df=DATAFRAMES['DF_SIMULATED']))

# You can also specify which percentile to pick values from 
print(metrics.slope_fdc(df=DATAFRAMES['DF_OBSERVED'], percentiles=(25, 77)))

            fdc_Slope
Station 1      3.1566
Station 2      2.3474
Station 3      2.1153
Station 4      4.3959
Station 5      4.5556
Station 6      3.3224
Station 7      7.9926
Station 8      7.0947
Station 9      1.5771
Station 10     4.1040
Station 11     7.0400
Station 12     2.3884
Station 13     5.6952
Station 14     2.7031
Station 15     2.6837
Station 16     2.6551
Station 17     5.6625
Station 18     5.5075
Station 19     1.1545
Station 20     1.3746
Station 21     1.5890
Station 22     6.1219
Station 23     0.7689
Station 24     0.7100
Station 25     6.3549
Station 26     1.0757
Station 27     4.4023
Station 28     3.1112
Station 29     4.9827
Station 30     1.2790
Station 31     5.9984
Station 32     4.5465
Station 33     3.5594
Station 34     3.4349
Station 35     1.6837
Station 36     5.7192
Station 37     1.0864
Station 38     0.7688
Station 39     5.2666
Station 40     1.2176
            fdc_Slope
Station 1      3.1056
Station 2      2.8474
Station 3      1.9309
Station 4 

### Comparison Metrics

These are the metrics that are used to compare the simulated and observed data. They work to show accurately we are able to predict the streamflow values using the models. Every other metric is a comparison metric. They are shown below:

- [Mean Square Error](#mean-square-error)
- [Root Mean Square Error](#root-mean-square-error)
- [Mean Average Error](#mean-average-error)
- [Nash-Sutcliffe Efficiency](#nash-sutcliffe-efficiency)
- [Kling-Gupta Efficiency](#kling-gupta-efficiency)
- [Percentage Bias](#percentage-bias)

#### Mean Square Error


In [9]:
# Mean square error for the data we were given
print(metrics.mse(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED']))

                model1
Station 1     1299.000
Station 2      780.600
Station 3       17.220
Station 4     5596.000
Station 5     5723.000
Station 6    16010.000
Station 7       85.470
Station 8      573.600
Station 9     1823.000
Station 10      45.170
Station 11      88.770
Station 12    1894.000
Station 13     522.400
Station 14    4731.000
Station 15    6572.000
Station 16    4964.000
Station 17     622.200
Station 18     123.100
Station 19    1417.000
Station 20    2296.000
Station 21    3495.000
Station 22     792.000
Station 23   11070.000
Station 24    1875.000
Station 25    1284.000
Station 26   13940.000
Station 27      54.710
Station 28       8.839
Station 29      55.620
Station 30   17770.000
Station 31      21.090
Station 32      93.040
Station 33     178.000
Station 34     191.900
Station 35   20800.000
Station 36      35.140
Station 37   25030.000
Station 38   74190.000
Station 39    2432.000
Station 40  142400.000


#### Root Mean Square Error

In [10]:
# Root Mean square error for the data we were given
print(metrics.rmse(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED']))

             model1
Station 1    36.050
Station 2    27.940
Station 3     4.150
Station 4    74.800
Station 5    75.650
Station 6   126.500
Station 7     9.245
Station 8    23.950
Station 9    42.690
Station 10    6.721
Station 11    9.422
Station 12   43.520
Station 13   22.860
Station 14   68.780
Station 15   81.070
Station 16   70.460
Station 17   24.940
Station 18   11.100
Station 19   37.640
Station 20   47.920
Station 21   59.120
Station 22   28.140
Station 23  105.200
Station 24   43.300
Station 25   35.840
Station 26  118.100
Station 27    7.397
Station 28    2.973
Station 29    7.458
Station 30  133.300
Station 31    4.592
Station 32    9.646
Station 33   13.340
Station 34   13.850
Station 35  144.200
Station 36    5.928
Station 37  158.200
Station 38  272.400
Station 39   49.320
Station 40  377.400


#### Mean Average Error

In [11]:
# Mean Average error for the data we were given
print(metrics.mae(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED']))

               model1
Station 1    209200.0
Station 2     29480.0
Station 3     15480.0
Station 4    505100.0
Station 5    526000.0
Station 6   1028000.0
Station 7     50180.0
Station 8    227700.0
Station 9    465300.0
Station 10    57960.0
Station 11    50780.0
Station 12   461400.0
Station 13   133100.0
Station 14   350900.0
Station 15   419200.0
Station 16   594000.0
Station 17   151200.0
Station 18    50810.0
Station 19   240700.0
Station 20   289900.0
Station 21   345900.0
Station 22   156500.0
Station 23   467200.0
Station 24   422800.0
Station 25   162300.0
Station 26   871900.0
Station 27    26880.0
Station 28     8420.0
Station 29    15500.0
Station 30   945600.0
Station 31    19470.0
Station 32    37040.0
Station 33    15620.0
Station 34    64230.0
Station 35   998700.0
Station 36    19900.0
Station 37  1109000.0
Station 38  2209000.0
Station 39   193900.0
Station 40  2784000.0


#### Nash-Sutcliffe Efficiency

In [12]:
# Nash-Sutcliffe Efficiency for the data we were given
print(metrics.nse(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED']))

             model1
Station 1   0.51660
Station 2  -1.67500
Station 3  -1.98200
Station 4   0.61250
Station 5   0.60570
Station 6   0.61670
Station 7   0.36630
Station 8   0.66540
Station 9   0.33660
Station 10 -0.38940
Station 11 -2.13300
Station 12  0.46520
Station 13  0.71100
Station 14  0.68990
Station 15  0.57330
Station 16  0.49380
Station 17  0.40150
Station 18 -0.15620
Station 19  0.58730
Station 20  0.47970
Station 21  0.32750
Station 22 -0.91590
Station 23 -0.75210
Station 24 -1.61800
Station 25  0.18760
Station 26  0.38680
Station 27 -0.87770
Station 28 -0.07323
Station 29 -2.48100
Station 30  0.31830
Station 31  0.32620
Station 32 -0.04744
Station 33  0.16150
Station 34 -0.35450
Station 35  0.35040
Station 36 -0.21350
Station 37  0.06404
Station 38  0.09380
Station 39 -4.97700
Station 40 -0.13280


##### Logarithm of the Nash-Sutcliffe Efficiency

In [13]:
# Logarithm of the Nash-Sutcliffe Efficiency for the data we were given
print(metrics.lognse(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED']))

              model1
Station 1   -0.25110
Station 2   -0.16920
Station 3   -0.01150
Station 4    0.11260
Station 5    0.11090
Station 6   -0.21200
Station 7   -0.47350
Station 8   -1.64900
Station 9   -1.84900
Station 10   0.03756
Station 11 -24.22000
Station 12  -1.99100
Station 13  -1.01700
Station 14  -0.03542
Station 15   0.03157
Station 16  -0.51460
Station 17  -0.01198
Station 18  -1.62300
Station 19   0.02779
Station 20   0.10010
Station 21   0.23950
Station 22  -1.44100
Station 23  -1.80400
Station 24  -7.49200
Station 25  -0.81920
Station 26  -0.96540
Station 27  -0.43530
Station 28  -0.14600
Station 29   0.04298
Station 30  -0.37720
Station 31  -1.15500
Station 32   0.05425
Station 33   0.16380
Station 34   0.45530
Station 35   0.18760
Station 36  -8.31800
Station 37   0.22880
Station 38   0.16550
Station 39  -0.66240
Station 40   0.28430


#### Kling-Gupta Efficiency

In [14]:
# Kling-Gupta Efficiency for the data we were given
print(metrics.kge(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED']))

             model1
Station 1   0.50940
Station 2  -0.11130
Station 3  -0.01851
Station 4   0.76400
Station 5   0.74670
Station 6   0.79720
Station 7   0.59780
Station 8   0.58520
Station 9   0.58250
Station 10  0.09986
Station 11 -0.22270
Station 12  0.61840
Station 13  0.72400
Station 14  0.77230
Station 15  0.69450
Station 16  0.70180
Station 17  0.51460
Station 18  0.37960
Station 19  0.75940
Station 20  0.69830
Station 21  0.65290
Station 22 -0.01509
Station 23  0.08618
Station 24  0.03342
Station 25  0.43380
Station 26  0.60250
Station 27  0.23020
Station 28  0.45020
Station 29 -0.16060
Station 30  0.62410
Station 31  0.24930
Station 32  0.24910
Station 33  0.27900
Station 34  0.39620
Station 35  0.63730
Station 36 -0.08113
Station 37  0.57010
Station 38  0.57380
Station 39 -0.81000
Station 40  0.45150


##### Modified Kling Gupta efficiency
This is different from the regular kge in that this uses the coefficient of Variation as its bias term (i.e., std/mean) as opposed to just the mean

In [15]:
# Kling-Gupta Efficiency for the data we were given
print(metrics.kge_2012(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED']))

             model1
Station 1   0.56060
Station 2   0.08006
Station 3  -0.20770
Station 4   0.73120
Station 5   0.71200
Station 6   0.81050
Station 7   0.31870
Station 8   0.36590
Station 9   0.11290
Station 10  0.11050
Station 11 -0.18180
Station 12  0.21030
Station 13  0.58420
Station 14  0.71620
Station 15  0.74840
Station 16  0.62130
Station 17  0.65220
Station 18 -0.19990
Station 19  0.66520
Station 20  0.65150
Station 21  0.58550
Station 22  0.17690
Station 23 -0.07924
Station 24  0.00923
Station 25  0.21240
Station 26  0.41710
Station 27 -0.11170
Station 28  0.49400
Station 29 -0.21180
Station 30  0.44990
Station 31  0.32740
Station 32 -0.01532
Station 33  0.05332
Station 34  0.22180
Station 35  0.52140
Station 36 -0.11350
Station 37  0.58590
Station 38  0.61420
Station 39 -0.07765
Station 40  0.42490


#### Percentage Bias

In [16]:
# Percentage Bias for the data we were given
print(metrics.bias(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED']))

            model1
Station 1   34.160
Station 2  -11.500
Station 3   13.790
Station 4  -13.500
Station 5  -16.580
Station 6   -2.373
Station 7   20.240
Station 8   38.900
Station 9   35.180
Station 10  48.370
Station 11  -2.078
Station 12  33.570
Station 13  23.840
Station 14   5.140
Station 15 -19.410
Station 16   7.604
Station 17 -11.770
Station 18  36.000
Station 19  10.930
Station 20  15.980
Station 21  14.410
Station 22 -11.270
Station 23   8.410
Station 24  36.100
Station 25  13.270
Station 26  13.650
Station 27  25.840
Station 28 -28.260
Station 29   3.039
Station 30  14.830
Station 31  49.710
Station 32  49.130
Station 33  51.860
Station 34  19.740
Station 35  10.120
Station 36  57.190
Station 37  -7.110
Station 38  -5.836
Station 39 -56.110
Station 40   2.214


Now that we have seen individual metrics, we also have the ability to calculate a list of metrics using our **calculate_all_metrics** or **calculate_metrics(list of merics)**. These are shown below:

In [17]:
metrices = ["MSE", "RMSE", "MAE", "NSE", "NegNSE"]
metrics.calculate_metrics(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED'],
                                            metrices=metrices)

,MSE,RMSE,MAE,NSE,NEGNSE
,model1,model1,model1,model1,model1
Station 1,1299.000,36.050,209200.0,0.51660,-0.51660
Station 2,780.600,27.940,29480.0,-1.67500,1.67500
Station 3,17.220,4.150,15480.0,-1.98200,1.98200
Station 4,5596.000,74.800,505100.0,0.61250,-0.61250
Station 5,5723.000,75.650,526000.0,0.60570,-0.60570
Station 6,16010.000,126.500,1028000.0,0.61670,-0.61670
Station 7,85.470,9.245,50180.0,0.36630,-0.36630
Station 8,573.600,23.950,227700.0,0.66540,-0.66540
Station 9,1823.000,42.690,465300.0,0.33660,-0.33660


We are also able to save these metrics as text files and csv files by specifying the **format** parameter and even the **out** parameter to specify a name to save it as.

In [18]:
metrics.calculate_all_metrics(observed=DATAFRAMES['DF_OBSERVED'], simulated=DATAFRAMES['DF_SIMULATED'],
                         format='txt', out='metrics'
                         )

See metrics.txt file in directory
